#### CRUD Com Tkinter

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import pyodbc
import pandas as pd

# Configuração do banco de dados
connection_string = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=sv-academy.database.windows.net;"
    "DATABASE=db-academy;"
    "UID=CloudSAa974ef83;"
    "PWD=IKAPl0[WPoi6;"
)
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# Funções CRUD
def create_transaction():
    try:
        query = f"""
        INSERT INTO dbo.projeto_daniel_danillo_matheus 
        (Tempo, Tipo, Valor, ContaOrigem, SaldoAntigoOrigem, SaldoNovoOrigem, ContaDestino, SaldoAntigoDestino, SaldoNovoDestino, Fraude)
        VALUES ({tempo.get()}, '{tipo.get()}', {valor.get()}, '{conta_origem.get()}', {saldo_antigo_origem.get()}, {saldo_novo_origem.get()},
                '{conta_destino.get()}', {saldo_antigo_destino.get()}, {saldo_novo_destino.get()}, {fraude.get()})
        """
        cursor.execute(query)
        conn.commit()
        messagebox.showinfo("Sucesso", "Transação criada com sucesso!")
    except Exception as e:
        messagebox.showerror("Erro", str(e))

def read_transactions():
    try:
        query = "SELECT * FROM dbo.projeto_daniel_danillo_matheus"
        user_condition = condition.get().strip()
        if user_condition:  # Se o usuário fornecer uma condição
            query += f" WHERE {user_condition}"

        df = pd.read_sql(query, conn)
        
        # Criar uma nova janela para mostrar os resultados
        result_window = tk.Toplevel(root)
        result_window.title("Resultados")
        text = tk.Text(result_window, wrap=tk.WORD)
        if not df.empty:
            text.insert(tk.END, df.to_string(index=False))
        else:
            text.insert(tk.END, "Nenhum resultado encontrado.")
        text.pack(fill=tk.BOTH, expand=True)
    except Exception as e:
        messagebox.showerror("Erro", str(e))

def update_transaction():
    try:
        update_str = f"{coluna.get()} = '{novo_valor.get()}'"
        query = f"UPDATE dbo.projeto_daniel_danillo_matheus SET {update_str} WHERE {condition.get()}"
        cursor.execute(query)
        conn.commit()
        messagebox.showinfo("Sucesso", "Transação atualizada com sucesso!")
    except Exception as e:
        messagebox.showerror("Erro", str(e))

def delete_transaction():
    try:
        query = f"DELETE FROM dbo.projeto_daniel_danillo_matheus WHERE {condition.get()}"
        cursor.execute(query)
        conn.commit()
        messagebox.showinfo("Sucesso", "Transação excluída com sucesso!")
    except Exception as e:
        messagebox.showerror("Erro", str(e))

# Configuração da janela principal
root = tk.Tk()
root.title("CRUD - Tkinter")
root.geometry("600x400")

# Aba principal
notebook = ttk.Notebook(root)
notebook.pack(fill=tk.BOTH, expand=True)

# Frame para cada aba
create_frame = ttk.Frame(notebook)
read_frame = ttk.Frame(notebook)
update_frame = ttk.Frame(notebook)
delete_frame = ttk.Frame(notebook)

notebook.add(create_frame, text="Create")
notebook.add(read_frame, text="Read")
notebook.add(update_frame, text="Update")
notebook.add(delete_frame, text="Delete")

# Componentes da aba "Create"
tk.Label(create_frame, text="Tempo").grid(row=0, column=0)
tempo = tk.Entry(create_frame)
tempo.grid(row=0, column=1)

tk.Label(create_frame, text="Tipo").grid(row=1, column=0)
tipo = tk.Entry(create_frame)
tipo.grid(row=1, column=1)

tk.Label(create_frame, text="Valor").grid(row=2, column=0)
valor = tk.Entry(create_frame)
valor.grid(row=2, column=1)

tk.Label(create_frame, text="Conta Origem").grid(row=3, column=0)
conta_origem = tk.Entry(create_frame)
conta_origem.grid(row=3, column=1)

tk.Label(create_frame, text="Saldo Antigo Origem").grid(row=4, column=0)
saldo_antigo_origem = tk.Entry(create_frame)
saldo_antigo_origem.grid(row=4, column=1)

tk.Label(create_frame, text="Saldo Novo Origem").grid(row=5, column=0)
saldo_novo_origem = tk.Entry(create_frame)
saldo_novo_origem.grid(row=5, column=1)

tk.Label(create_frame, text="Conta Destino").grid(row=6, column=0)
conta_destino = tk.Entry(create_frame)
conta_destino.grid(row=6, column=1)

tk.Label(create_frame, text="Saldo Antigo Destino").grid(row=7, column=0)
saldo_antigo_destino = tk.Entry(create_frame)
saldo_antigo_destino.grid(row=7, column=1)

tk.Label(create_frame, text="Saldo Novo Destino").grid(row=8, column=0)
saldo_novo_destino = tk.Entry(create_frame)
saldo_novo_destino.grid(row=8, column=1)

tk.Label(create_frame, text="Fraude").grid(row=9, column=0)
fraude = tk.Entry(create_frame)
fraude.grid(row=9, column=1)

tk.Button(create_frame, text="Criar Transação", command=create_transaction).grid(row=10, column=0, columnspan=2)

# Componentes da aba "Read"
tk.Label(read_frame, text="Condição (opcional)").pack()
condition = tk.Entry(read_frame)
condition.pack()

tk.Button(read_frame, text="Ler Transações", command=read_transactions).pack()

# Componentes da aba "Update"
tk.Label(update_frame, text="Coluna").pack()
coluna = tk.Entry(update_frame)
coluna.pack()

tk.Label(update_frame, text="Novo Valor").pack()
novo_valor = tk.Entry(update_frame)
novo_valor.pack()

tk.Label(update_frame, text="Condição").pack()
condition = tk.Entry(update_frame)
condition.pack()

tk.Button(update_frame, text="Atualizar Transação", command=update_transaction).pack()

# Componentes da aba "Delete"
tk.Label(delete_frame, text="Condição").pack()
condition = tk.Entry(delete_frame)
condition.pack()

tk.Button(delete_frame, text="Excluir Transação", command=delete_transaction).pack()

# Iniciar o loop principal 0000000
root.mainloop()

C:\Users\BlueShift\AppData\Local\Temp\ipykernel_6860\104976137.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


: 